In [1]:
import googleapiclient.discovery
import spacy
from spacy_langdetect import LanguageDetector
import pandas as pd
from spacy.language import Language
import re
from spacytextblob.spacytextblob import SpacyTextBlob
from textblob import TextBlob
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import emoji

import environ

from utils.def_api_google import obtenir_infos_video_et_commentaires

In [2]:
env = environ.Env()
environ.Env().read_env('.env')

API_KEY = env('API_KEY')

In [ ]:
videos_youtubeur_1 = ['woiFO4o3CXI',
'7_60w7T1Oak',
'coWT9YsAAC0',
'L9HzH8v0Now',
'N-TCJquxeFk',
'qSUE11N1GPM',
'8jluurK-E6U',
'LyDJTp4nS3w',
'LBraVzyYDKg',
'tCIIrPEF8Hk']

videos_youtubeur_2 = ['bbetYpYH-gI',
'iU34s9rbPoE',
'x2WASA8bb-I',
'JbiBkKy4NFs',
'GFnzjrGh3Yc',
'XhbECIm1Wa4',
'rKRAkNc4OKw',
'7a0DayJnGMc',
'V9dJEWTjxlM',
'dcTAk9ASfIk']

In [ ]:
# On récupère les commentaires des vidéos
for i in videos_youtubeur_1:
    commentaires_1, infos_video_1 = obtenir_infos_video_et_commentaires(i, API_KEY)  
    print(i)

# for i in videos_youtubeur_2:
#     commentaires_2, infos_video_2 = obtenir_infos_video_et_commentaires(i, API_KEY)

woiFO4o3CXI
7_60w7T1Oak
coWT9YsAAC0
L9HzH8v0Now
N-TCJquxeFk
qSUE11N1GPM
8jluurK-E6U
LyDJTp4nS3w
